In [58]:
import geopandas
import urllib
import shapely
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians
import json
from PIL import Image
from dotenv import load_dotenv
import os
import pandas as pd
from tqdm import tqdm_notebook
import time

load_dotenv()

True

In [69]:
def getDistance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def getCitySize(cityBounds):
    lat1, lon1, lat2, lon2 = cityBounds
    return getDistance(lat1, lon1, lat2, lon2)

def getGoogleBackgroundMap(cityBounds, size="500x500", maptype="terrain", scale=2):
    maps_key = os.getenv("GOOGLE_MAPS_API")
    staticmap_base_url = 'https://maps.googleapis.com/maps/api/staticmap'
    
    topLeftLat, topLeftLong, botRightLat, botRightLong = cityBounds
    pathCode = f"color:0x0000ff00|{botRightLong},{topLeftLat}|{topLeftLong},{botRightLat}"
    
    url = staticmap_base_url + '?' + urllib.parse.urlencode({
                "size": size,
                "maptype": maptype,
                "scale": scale,
                "key": maps_key,
                "path": pathCode
            })
    image = urllib.request.urlopen(url).read()
    return image

def getCityOutline(query: str):
    openStreetMap_base_url = "https://nominatim.openstreetmap.org/search"
    url = openStreetMap_base_url + '?' + urllib.parse.urlencode({
        "q": query,
        "format": "json",
        "polygon_geojson": 1
    })

    response = urllib.request.urlopen(url).read()
    responseJson = json.loads(response)

    # find first multipolygon/polygon result (most likely the correct one)
    for shape in responseJson:
        # Included this try if shape has no geojson. Now just goes to next entry
        try:
            if isinstance(shapely.geometry.shape(shape["geojson"]), shapely.geometry.MultiPolygon):
                cityShape = shapely.geometry.shape(shape["geojson"])
                lat1, long1, lat2, long2 = cityShape.bounds
                bounds = [lat1, long1, lat2, long2]
                # check if size is way too big or too small. If so --> kick out
                size = getCitySize(bounds)
                if size > 200 or size < 5:
                    continue
                cityGDF = geopandas.GeoDataFrame({
                    "geometry": cityShape
                })
                return cityGDF, bounds
            elif isinstance(shapely.geometry.shape(shape["geojson"]), shapely.geometry.Polygon):
                cityShape = shapely.geometry.shape(shape["geojson"])
                lat1, long1, lat2, long2 = cityShape.bounds
                bounds = [lat1, long1, lat2, long2]
                # check if size is way too big or too small. If so --> kick out
                size = getCitySize(bounds)
                if size > 200 or size < 5:
                    continue
                cityGDF = geopandas.GeoDataFrame({
                    "geometry": cityShape
                }, index=[0])
                return cityGDF, bounds
        except:
            continue
    # If nothing found return None
    return None, None

def getGoogleBackgroundMap(cityBounds, size="500x500", maptype="satellite", scale=2):
    maps_key = os.getenv("GOOGLE_MAPS_API")
    staticmap_base_url = 'https://maps.googleapis.com/maps/api/staticmap'
    
    topLeftLat, topLeftLong, botRightLat, botRightLong = cityBounds
    pathCode = f"color:0x0000ff00|{botRightLong},{topLeftLat}|{topLeftLong},{botRightLat}"
    
    url = staticmap_base_url + '?' + urllib.parse.urlencode({
                "size": size,
                "maptype": maptype,
                "scale": scale,
                "key": maps_key,
                "path": pathCode
            })
    image = urllib.request.urlopen(url).read()
    return image

def getImages(query: str):
    # Get outline from openstreetmaps and save in outline-folder
    queryFilename = query.replace(", ","_")
    queryFilename = queryFilename.replace(",","_")
    cityShape, bounds = getCityOutline(query)
    if cityShape is None:
        return False
    cityShape.plot()
    plt.axis("off")
    plt.savefig(f"data/outlines/outline_{queryFilename}.png", transparent=True)
    plt.close()
    # use google maps with , but invisible ink (satellite) and save in sat-image folder 
    sat_im = getGoogleBackgroundMap(bounds)
    with open(f"data/sat_images/sat_{queryFilename}.png", "wb") as f:
        f.write(sat_im)
    # save them accordingly
    return True

In [29]:
# Load simplemaps data and get cities
cityData = pd.read_excel("data/worldcities.xlsx")


In [70]:
# Loop over the cities
# No need to load duplicates
print(cityData.columns)
highPopCities = cityData[cityData["population"]> 500000][["city_ascii","country"]]
# print(highPopCities)
countSuccessfull = 0
for i, cityInd in tqdm_notebook(highPopCities.iterrows(), total=highPopCities.shape[0]):
    query = ",".join(cityInd.values)
    queryFilename = "_".join(cityInd.values)

    # check if already exists. If yes --> skip
    if os.path.exists(f"data/outlines/outline_{queryFilename}.png"):
        continue
    else:
        if getImages(query):
            countSuccessfull += 1
    # Sleep for a second to not get banned from OpenStreetmaps
    time.sleep(1)
    
print(f"Successful images loaded: {countSuccessfull} from {highPopCities.shape[0]} ({countSuccessfull/highPopCities.shape[0]*100:5.2f}%)")

Index(['city', 'city_ascii', 'lat', 'lng', 'country', 'iso2', 'iso3',
       'admin_name', 'capital', 'population', 'id'],
      dtype='object')


Successful images loaded: 693 from 974 (71.15%)
